In [1]:
import torch
from data_handlers import Load_ImageNet100
from overcomplete.models import DinoV2, ViT, ResNet
from torch.utils.data import DataLoader, TensorDataset
import os
from torch.utils.data import Dataset, DataLoader
import glob

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
models = {
    "DinoV2": DinoV2(device="cuda"),
    "ViT": ViT(device="cuda")
}

save_dir = "activations/ImageNet100_Dino_ViT"
dataloader = Load_ImageNet100(transform=None, batch_size=256, shuffle=True)
dataloader = Load_ImageNet100(transform=model.preprocess, batch_size=256, shuffle=True)

Using cache found in C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main
C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
C:\Users\sproj_ha/.cache\torch\hub\facebookresearch_dinov2_main\dinov2\layers\block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
c:\Users\sproj_ha\miniconda3\envs\interpretability\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'model' is not defined

In [5]:
os.makedirs(save_dir, exist_ok=True)
for key, model in models.items():
    model.to(device).eval()

with torch.no_grad():
    for i, (images, _) in enumerate(dataloader):
        images = images.to(device)
        batch_data = {'images': images.cpu()}
        
        for key, model in models.items():
            activations = model.forward_features(images)
            batch_data[f"activations_{key}"] = activations.cpu()

        torch.save(batch_data, os.path.join(save_dir, f"batch_{i:05d}.pt"))
        print(f"Saved batch {i} with {images.size(0)} samples.")


Saved batch 0 with 256 samples.
Saved batch 1 with 256 samples.
Saved batch 2 with 256 samples.
Saved batch 3 with 256 samples.
Saved batch 4 with 256 samples.
Saved batch 5 with 256 samples.
Saved batch 6 with 256 samples.
Saved batch 7 with 256 samples.
Saved batch 8 with 256 samples.
Saved batch 9 with 256 samples.
Saved batch 10 with 256 samples.
Saved batch 11 with 256 samples.
Saved batch 12 with 256 samples.
Saved batch 13 with 256 samples.
Saved batch 14 with 256 samples.
Saved batch 15 with 256 samples.
Saved batch 16 with 256 samples.
Saved batch 17 with 256 samples.
Saved batch 18 with 256 samples.
Saved batch 19 with 256 samples.
Saved batch 20 with 256 samples.
Saved batch 21 with 256 samples.
Saved batch 22 with 256 samples.
Saved batch 23 with 256 samples.
Saved batch 24 with 256 samples.
Saved batch 25 with 256 samples.
Saved batch 26 with 256 samples.
Saved batch 27 with 256 samples.
Saved batch 28 with 256 samples.
Saved batch 29 with 256 samples.
Saved batch 30 with 

In [11]:

class PTFilesDataset(Dataset):
    def __init__(self, directory_path):
        """
        Args:
            directory_path (string): Path to the directory containing .pt files
        """
        self.file_paths = sorted(glob.glob(os.path.join(directory_path, '*.pt')))
        
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        # Load the dictionary from the .pt file
        data_dict = torch.load(self.file_paths[idx])
        
        # Assuming the dictionary contains tensors that can be directly used
        # You might need to modify this part based on your specific data structure
        return data_dict

dataset = PTFilesDataset(directory_path='./activations/ImageNet100_Dino_ViT')

# Create dataloader
dataloader = DataLoader(
dataset,
batch_size=1,          # Adjust batch size as needed
shuffle=True,           # Shuffle if training
drop_last=True        # Whether to drop last incomplete batch
)

# Iterate through the dataloader
for batch_idx, batch_data in enumerate(dataloader):
    # batch_data will be a dictionary with batched tensors
    # Each key in the original dictionary will now have a tensor with batch_size as first dim
    print(f"Batch {batch_idx}, data keys: {batch_data.keys()}")
    # Your training/validation code here

Batch 0, data keys: dict_keys(['images', 'activations_DinoV2', 'activations_ViT'])
Batch 1, data keys: dict_keys(['images', 'activations_DinoV2', 'activations_ViT'])
Batch 2, data keys: dict_keys(['images', 'activations_DinoV2', 'activations_ViT'])
Batch 3, data keys: dict_keys(['images', 'activations_DinoV2', 'activations_ViT'])
Batch 4, data keys: dict_keys(['images', 'activations_DinoV2', 'activations_ViT'])
Batch 5, data keys: dict_keys(['images', 'activations_DinoV2', 'activations_ViT'])
Batch 6, data keys: dict_keys(['images', 'activations_DinoV2', 'activations_ViT'])
Batch 7, data keys: dict_keys(['images', 'activations_DinoV2', 'activations_ViT'])
Batch 8, data keys: dict_keys(['images', 'activations_DinoV2', 'activations_ViT'])
Batch 9, data keys: dict_keys(['images', 'activations_DinoV2', 'activations_ViT'])
Batch 10, data keys: dict_keys(['images', 'activations_DinoV2', 'activations_ViT'])
Batch 11, data keys: dict_keys(['images', 'activations_DinoV2', 'activations_ViT'])
Ba